## **Machine Vision Project**
### **Object detection in radar images (ship detection)**

### **Project objective:**

The main intention behind this project is to develop a tool capable of identifying ships from radar imagery, either in static images or video sequences. The objective is to design and evaluate an object detection pipeline that can automatically locate and classify ships within complex radar data, where noise, low contrast, and environmental interference are common challenges.

This goal is achieved through the development of a recognition system implemented in Python, using a Jupyter Notebook as the main experimentation and documentation environment. The notebook format allows for an iterative workflow where data preprocessing, model training, evaluation, and visualization coexist in a single, transparent space.

### **Motivation**

Maritime surveillance is a critical task in areas such as port security, border control, illegal fishing detection, and collision avoidance systems. Radar imagery plays a key role in these scenarios due to its robustness under adverse weather conditions and low-visibility environments, where traditional optical sensors often fail. However, interpreting radar images manually is time-consuming and prone to human error.

By applying machine vision techniques to radar data, this project explores how automated systems can assist or replace manual monitoring, improving efficiency, scalability, and response time. In short: less eyeballing blips, more intelligence.

### **Approach Overview**

The project follows a classical machine vision pipeline adapted to radar imagery:

- Data exploration and preprocessing to reduce noise and enhance relevant features

- Annotation and preparation of training data

- Application of object detection techniques (traditional or learning-based, depending on constraints)

- Model evaluation using appropriate metrics to assess detection accuracy and robustness

Special attention is given to the unique characteristics of radar images, such as speckle noise, varying signal intensity, and ambiguous object boundaries.

### **Scope and Limitations**

The scope of this project is limited to ship detection within the provided dataset and does not aim to perform ship tracking or identification beyond basic object detection. Additionally, performance may vary depending on radar resolution, environmental conditions, and dataset diversity. These limitations are acknowledged and discussed as part of the final evaluation.




### First quick checks

These checks are only for quick verifications so everything runs as smoothly as possible in the notebook and the execution enviroment, which in this case it is Google Collab.

In [ ]:
# (Recommended) Run on a GPU runtime
import os, sys, platform
print('Python:', sys.version)
print('Platform:', platform.platform())

# Optional: quick CUDA check (PyTorch may not be installed yet)
try:
    import torch
    print('Torch:', torch.__version__)
    print('CUDA available:', torch.cuda.is_available())
    if torch.cuda.is_available():
        print('GPU:', torch.cuda.get_device_name(0))
except Exception as e:
    print('Torch not available yet (will install below).')

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
Torch: 2.9.0+cpu
CUDA available: False


## Installing necessary dependencies

In [ ]:
# Install dependencies
# Ultralytics provides YOLO training/inference; pycocotools reads COCO JSON; shapely helps geometry.

!pip -q install --upgrade ultralytics
!pip -q install pycocotools shapely opencv-python matplotlib tqdm pyyaml

# Restart runtime if Ultralytics asks you to.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.2 MB/s eta 0:00:00


## 1) Get the HRSID dataset

HRSID images are large; downloading may take a while.

You have two options:

### Option A — Manual download (recommended)
1. Download HRSID (JPG) from the links in the official repository.
2. Extract it into a folder like:

```
./HRSID_ROOT/
  images/...
  annotations/...
```

### Option B — Programmatic download (Google Drive)
If the dataset is hosted on Google Drive, you can often download it with `gdown` using the file id.

> If your download fails (quota, permissions), use manual download.

In [3]:
from pathlib import Path

# Point this to where you placed/extracted the dataset
HRSID_ROOT = Path('HRSID_ROOT')  # <-- change me

# Expected subfolders (adjust if your extraction differs)
ANNOT_DIR = HRSID_ROOT / 'annotations'
IMG_DIR = HRSID_ROOT / 'images'

print('HRSID_ROOT:', HRSID_ROOT.resolve())
print('ANNOT_DIR exists:', ANNOT_DIR.exists())
print('IMG_DIR exists:', IMG_DIR.exists())

# List files (helps you discover exact structure)
if ANNOT_DIR.exists():
    print('Annotation files:', sorted([p.name for p in ANNOT_DIR.glob('*.json')])[:10])
if IMG_DIR.exists():
    exts = ['*.jpg','*.jpeg','*.png']
    imgs = []
    for e in exts:
        imgs += list(IMG_DIR.rglob(e))
    print('Found images:', len(imgs))

HRSID_ROOT: /content/HRSID_ROOT
ANNOT_DIR exists: False
IMG_DIR exists: False


In [4]:
from pathlib import Path

# Set these based on what exists in your annotations folder.
COCO_TRAIN_JSON = None
COCO_VAL_JSON = None
COCO_TEST_JSON = None

# Auto-detect common names
cands = list((ANNOT_DIR).glob('*.json')) if ANNOT_DIR.exists() else []
name2path = {p.name.lower(): p for p in cands}

for key in ['train.json','train2017.json','hrsid_train.json','instances_train.json']:
    if key in name2path:
        COCO_TRAIN_JSON = name2path[key]
        break

for key in ['val.json','valid.json','hrsid_val.json','instances_val.json','val2017.json']:
    if key in name2path:
        COCO_VAL_JSON = name2path[key]
        break

for key in ['test.json','hrsid_test.json','instances_test.json']:
    if key in name2path:
        COCO_TEST_JSON = name2path[key]
        break

# If no train JSON found, fall back to the first json in the folder
if COCO_TRAIN_JSON is None and cands:
    COCO_TRAIN_JSON = cands[0]

print('COCO_TRAIN_JSON:', COCO_TRAIN_JSON)
print('COCO_VAL_JSON:', COCO_VAL_JSON)
print('COCO_TEST_JSON:', COCO_TEST_JSON)

COCO_TRAIN_JSON: None
COCO_VAL_JSON: None
COCO_TEST_JSON: None
